# Problem 1 #

Using the data from table 2-3 from D&H page 64, compute six group and total delayed neutron fraction for $^{235}$U thermal fission.

In [19]:
import numpy as np
import pandas as pd

In [20]:
delayed_neutrons_per_thermal_fission = 0.01668
neutrons_per_thermal_fission = 2.43
delayed_group = pd.DataFrame(
    {
        "Group": range(1,7),
        "Half Life": [54.51, 21.84, 6.00, 2.23, 0.496, 0.179],
        "Relative Yield": [0.038, 0.213, 0.188, 0.407, 0.128, 0.026]
    }
)
delayed_group.set_index("Group", inplace=True)
delayed_group

,Half Life,Relative Yield
Group,,
1,54.510,0.038
2,21.840,0.213
3,6.000,0.188
4,2.230,0.407
5,0.496,0.128
6,0.179,0.026


In [21]:
delayed_group["Delayed Neutron Fraction"] =\
    delayed_group["Relative Yield"] * delayed_neutrons_per_thermal_fission / neutrons_per_thermal_fission
delayed_group

,Half Life,Relative Yield,Delayed Neutron Fraction
Group,,,
1,54.510,0.038,0.000261
2,21.840,0.213,0.001462
3,6.000,0.188,0.001290
4,2.230,0.407,0.002794
5,0.496,0.128,0.000879
6,0.179,0.026,0.000178


In [22]:
print "Total delayed neutron fraction: {0}".format(sum(delayed_group["Delayed Neutron Fraction"]))

Total delayed neutron fraction: 0.00686419753086


In [23]:
# verification
(0.01668) / (2.43 + 0.01668)

0.006817401540046103

# Problem 2 #

Repeate the solution of the point reactor kinetics equation with one effective delayed group for a constant reactivity addition $\rho_0$ using the Laplace transform method.

$$
\dot{N(t)} = \left( \frac{\rho_0 - \beta}{\Lambda} \right) N(t) + \lambda C(t) \\
\dot{C(t)} = \frac{\beta}{\Lambda} N(t) - \lambda C(t)
$$

Here we perform our Laplace transformation:

$$
s \tilde{N(s)} - N_0 = \frac{\rho_0 - \beta}{\Lambda} \tilde{N(s)} + \lambda \tilde{C(s)} \\
s \tilde{C(s)} - C_0 = \frac{\beta}{\Lambda} \tilde{N(s)} - \lambda \tilde{C(s)}
$$

$$
(s+\lambda)\tilde{C}=\frac{\beta}{\Lambda}\tilde{N}+C_{0} \\
$$
$$
\tilde{C}=\frac{\beta}{\Lambda(s+\lambda)}\tilde{N}+\frac{C_{0}}{s+\lambda} \\
$$
$$
(s-\frac{{\rho_{0}-\beta}}{\Lambda})\tilde{N}=\lambda\tilde{C}+N_{0} \\
$$
$$
(s-\frac{{\rho_{0}-\beta}}{\Lambda})\tilde{N}=\lambda(\frac{\beta}{\Lambda(s+\lambda)}\tilde{N}+\frac{C_{0}}{s+\lambda})+N_{0} \\
$$
$$
C_{0}=\frac{\beta}{\lambda\Lambda}N_{0} \\
$$
$$
(s-\frac{{\rho_{0}-\beta}}{\Lambda})\tilde{N}=\lambda(\frac{\beta}{\Lambda(s+\lambda)}\tilde{N}+\frac{\beta N_{0}}{\lambda\Lambda(s+\lambda)})+N_{0} \\
$$
$$
(s\Lambda-\rho_{0}-\beta)\tilde{N}=\lambda(\frac{\beta}{(s+\lambda)}\tilde{N}+\frac{\beta N_{0}}{\lambda(s+\lambda)})+N_{0}\Lambda \\
$$
$$
((s\Lambda-\rho_{0}-\beta)-\frac{\lambda\beta}{s+\lambda})\tilde{N}=(\frac{\beta}{s+\lambda}+\Lambda)N_{0} \\
$$
$$
\frac{\tilde{N}}{N_{0}}=((s\Lambda-\rho_{0}-\beta)-\frac{\lambda\beta}{s+\lambda})^{-1}(\frac{\beta}{s+\lambda}+\Lambda) \\
$$

# Problem 3 #

A PWR filled with $^{235}$U is initially operating at a power level of 1000 MWt. What is the power of the reactor after 60 seconds for reactivity insertions of -20 cents and +20 cents?

Assume a prompt neutron lifetime of 0.1 milliseconds

In [24]:
tau_prompt = 1e-4

In [25]:
delayed_group["Decay Constant"] = np.log(2.) / delayed_group["Half Life"]
delayed_group.ix[:, delayed_group.columns.difference(["Relative Yield"])]

,Decay Constant,Delayed Neutron Fraction,Half Life
Group,,,
1,0.012716,0.000261,54.510
2,0.031738,0.001462,21.840
3,0.115525,0.001290,6.000
4,0.310828,0.002794,2.230
5,1.397474,0.000879,0.496
6,3.872331,0.000178,0.179


In [26]:
beta = sum(delayed_group["Delayed Neutron Fraction"])

In [27]:
theta = delayed_group["Delayed Neutron Fraction"].dot(delayed_group["Half Life"]) + (1. - beta) * tau_prompt
theta

0.060689942123456783

In [28]:
print 'beta: {0}'.format(beta)
print "theta: {0}".format(theta)

beta: 0.00686419753086
theta: 0.0606899421235


In [29]:
def k(reactivity):
    return 1. / (1. - reactivity)

def tau(theta, k):
    return theta / np.log(k)

In [30]:
def power(k, t, initial=1000):
    period = tau(theta, k)
    p = initial * np.exp(t / period)
    
    return p

In [31]:
power_plus_twenty_cents = power(k(0.2 * beta), 60)
power_plus_twenty_cents

3889.0508282555074

In [32]:
power_minus_twenty_cents = power(k(-0.2 * beta), 60)
power_minus_twenty_cents

257.61170155452254

# Problem 4 #

Using the data in Table 2-3 from D&H page 64 and Figure 2-20 from D&H page 61 for $^{235}$U solve the inhour equation below to obtain the decay constants, $s$, for reactivity values of \$-0.20 and \$+0.20.

\begin{equation}
    \rho = \Lambda s + \sum_{i=1}^6 \frac{\beta_i s}{s + \lambda_i}
\end{equation}

Using the reactivity function defined in number 3 and $\Lambda = \frac{\ell}{k}$

In [67]:
from sympy import Symbol
from sympy.solvers import solve
from sympy import init_printing, latex

In [61]:
init_printing()

In [62]:
s = Symbol('s')

In [63]:
reactivity = -.2 * beta

In [64]:
solve(
    s * 1e-4 / k(reactivity) \
        + sum(delayed_group["Delayed Neutron Fraction"] * s / (s + delayed_group["Decay Constant"])) \
        - reactivity, 
    s
)

In [65]:
reactivity = 0.2

In [66]:
solve(
    s * 1e-4 / k(reactivity) \
        + sum(delayed_group["Delayed Neutron Fraction"] * s / (s + delayed_group["Decay Constant"])) \
        - reactivity,
    s
)